In [46]:
.ORIG x3000
.SET MODE TERMINAL
    ST     R7, REG7
    JSR PUSH 
    ADD R5, R5, #4
    ADD R1, R1, #13
    JSR GETCHOICE
    TRAP x22
    AND R0, #0
    TRAP x23
    JSR GETSTEP
    ADD R5, R5, #0
    BRp #-7
    JSR POP        ;restore the value for all registers
    LD  R7, REG7
    HALT
     
PUSH:  ST R6, REG6     ;; save R5, R6 to MEMORY
       ST R5, REG5     ;; R7 has return address
                       ;; REG7 is in MEMORY already
                       ;; R6 and R5 are destroyed
       LD R6, SREG     ;; R6 is now TOP of stack
       STR R0, R6, #0      ;; push R0 on stack
       ADD R6, R6, #1  ;; incr
       STR R1, R6, #0      ;; push R1 on stack
       ADD R6, R6, #1  ;; incr
       STR R2, R6, #0      ;; push R2 on stack
       ADD R6, R6, #1  ;; incr
       STR R3, R6, #0      ;; push R3 on stack
       ADD R6, R6, #1  ;; incr
       STR R4, R6, #0      ;; push R4 on stack
       ADD R6, R6, #1  ;; incr
       STR R5, R6, #0      ;; push R5 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG6     ;; get R6 from MEMORY (destroys R5)
       STR R5, R6, #0      ;; push R6 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG7     ;; get R7 from MEMORY
       STR R5, R6, #0  ;; push R7 on stack
       ADD R6, R6, #1  ;; incr
       ST R6, SREG      ;; save incremented SREG
       LD R6, REG6
       LD R5, REG5     ;;
       RET
     
POP:   LD R6, SREG     ;; R6 is TOP of stack
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R7
       ST  R1, REG7    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R6
       ST  R1, REG6    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R5, R6, #0      ;; load R5
       ADD R6, R6, #-1     ;; decr
       LDR  R4, R6, #0      ;; load R4
       ADD R6, R6, #-1     ;; decr
       LDR  R3, R6, #0      ;; load R3
       ADD R6, R6, #-1     ;; decr
       LDR  R2, R6, #0      ;; load R2
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R1
       ADD R6, R6, #-1     ;; decr
       LDR  R0, R6, #0      ;; load R0
       ST R6, SREG      ;; save decremented TOP
       LD R6, REG6, #0     ;; put R6 back the way it was
       RET


REG:    .BLKW    #8
SREG:   .FILL REG
REG5:   .FILL    #0
REG6:   .FILL    #0
REG7:   .FILL    #0
HALT     
     
GETCHOICE: AND R0, #0
           LEA R0, ASKFORCHOICE
           RET
           
;;R1 -- UPDATE THE RESULT
;;R5 -- STEP LIMIT
;;R6 -- ADDRESS IN R7(return address)
;;R2 -- GOAL

GETSTEP: ;;;;;;;;;;;;;;;;;;;;start pushing
         LD R6, REG7_TOP
         STR R7, R6, #0
         ;;;;;;;;;;;;;;;;;;;;Finish pushing R7 on stack
         ADD R0, R0, #-15
         ADD R0, R0, #-15
         ADD R0, R0, #-15
         ADD R0, R0, #-4
         BRz OPERATION1
         ADD R0, R0, #-1
         BRz OPERATION2
         ADD R0, R0, #-1
         BRz OPERATION3
         ;;;;;;;;;;;;;;;;;;;; start poping
         ADD R6, R6, #-1
         LDR R7, R6, #0
         ;;;;;;;;;;;;;;;;;;;;Finish poping
         RET

OPERATION1: ;;;;;;;;;;;;;;;;; operations(need to change @Eileen)
            ADD R1, R1, #-1
            ;;;;;;;;;;;;;;;;; start stack
            ADD R6, R6, #1
            STR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack
            JSR CHECKSTEP
            ADD R6, R6, #-1
            LDR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish poping 
            RET

OPERATION2: ;;;;;;;;;;;;;;;;; start stack
            ADD R6, R6, #1
            STR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack
            JSR DIVTWO
            JSR CHECKSTEP
            ADD R6, R6, #-1
            LDR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish poping 
            RET

DIVTWO: ADD R2, R2, #-2  ; ALWAYS USE R2 TO STORE THE DENOMITOR
        ADD R3, R3, R1   ; R3 to store the current R1
        ADD R4, R4, #0   ; R4 to store the result of calculating
        ADD R1, R1, R2   ; minus R1 by 2
        BRn #4
        BRz #9
        ADD R4, R4, #1
        ADD R1, R1, #0
        BRp #-6
        AND R1, R1, #0
        ADD R1, R1, R3  ; restore the value of R1 if not dividable
        AND R2, R2, #0
        AND R3, R3, #0
        AND R4, R4, #0
        RET
        ADD R1, R1, R4  ; otherwise udpate the value of R1
        ADD R1, R1, #1
        AND R2, R2, #0
        AND R3, R3, #0
        AND R4, R4, #0
        RET

OPERATION3: ;;;;;;;;;;;;;;;;; start stack
            ADD R6, R6, #1
            STR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack
            JSR DIVTHREE
            JSR CHECKSTEP
            ADD R6, R6, #-1
            LDR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish poping 
            RET

DIVTHREE:   ADD R2, R2, #-3  ; ALWAYS USE R2 TO STORE THE DENOMITOR
            ADD R3, R3, R1   ; R3 to store the current R1
            ADD R4, R4, #0   ; R4 to store the result of calculating
            ADD R1, R1, R2   ; minus R1 by 2
            BRn #4
            BRz #9
            ADD R4, R4, #1
            ADD R1, R1, #0
            BRp #-6
            AND R1, R1, #0
            ADD R1, R1, R3  ; restore the value of R1 if not dividable
            AND R2, R2, #0
            AND R3, R3, #0
            AND R4, R4, #0
            RET
            ADD R1, R1, R4  ; otherwise udpate the value of R1
            ADD R1, R1, #1
            AND R2, R2, #0
            AND R3, R3, #0
            AND R4, R4, #0
            RET
            
CHECKSTEP: ;;;;;;;;;;;;;;;;;; update step
           ADD R6, R6, #1
           ADD R5, R5, #-1
           BRp #5
           ;;;;;;;;;;;;;;;;;; start stack
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           ADD R5, R5, #0
           BRz FINALSTATE
           ;;;;;;;;;;;;;;;;;; start poping
           ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           RET

FINALSTATE:;;;;;;;;;;;;;;;;;; when step limit is consumed 
           ;;;;;;;;;;;;;;;;; start stack
           ADD R6, R6, #1
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           LD R2, GOAL
           NOT R2, R2
           ADD R2, R2, #1
           ADD R3, R1, R2
           BRp #3
           BRz WIN_STATE
           ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           ;;JSR LOSE_STATE
           RET

LOSE_STATE: RET 

WIN_STATE: ;;;;;;;;;;;;;;;;; start stack
           ADD R6, R6, #1
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           JSR WIN_MES
           TRAP x22
           ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           ;;;;;;;;;;;;;;;;; start stack
           ADD R6, R6, #1
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           AND R0, #0
           JSR WIN_PIC
           TRAP x22
           ;ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           HALT
    
WIN_MES: AND R0, #0
         LEA R0, WIN_M
         RET
         
WIN_PIC: AND R0, #0
         LEA R0, HEADER
         TERMINAL R0, 0
         RET
         
REG7_STACK: .BLKW #10
;REG6: .FILL #0
REG7_TOP: .FILL REG7_STACK
TEMP: .FILL #0
TEMP2: .FILL #0
TEMP3: .FILL #0
GOAL: .FILL #1
ASKFORCHOICE: .STRINGZ "Please enter your choice(1 or 2 or 3):"
GAMEOPS: .STRINGZ "(1)A -= 1    (2)A /= 2 (if A%2 == 0)    (3)A =/ 3(if A%3 == 0)\n"
INITIAL: .STRINGZ "12"
WIN_M: .STRINGZ "\nYOU WIN!"
HEADER: .STRINGZ "<img width = '400' src='https://raw.githubusercontent.com/EileenFeng/240FinalProject/master/Camille%20Monet%20on%20a%20Garden%20Bench.jpg'>"    
.END 

Assembled! Use %dis or %dump to examine; use %exe to run.


In [47]:
%exe

Please enter your choice(1 or 2 or 3):
Input a character> 1
1
Please enter your choice(1 or 2 or 3):
Input a character> 2
2
Please enter your choice(1 or 2 or 3):
Input a character> 3
3
Please enter your choice(1 or 2 or 3):
Input a character> 1
1

YOU WIN!

<img width = '400' src='https://raw.githubusercontent.com/EileenFeng/240FinalProject/master/Camille%20Monet%20on%20a%20Garden%20Bench.jpg'>============================================================
Computation completed
Instructions: 4624
Cycles: 37990 (0.018995 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x3146 R1: x0001 R2: xFFFF R3: x0000 
R4: x0000 R5: x0000 R6: x30C7 R7: x30BC 


In [48]:
%dis

Memory disassembled:
           x3000: x3E47  ST R7, REG7                               [line: 2]
           x3001: x480C  JSR PUSH                                  [line: 3]
           x3002: x1B64  ADD R5, R5, #4                            [line: 4]
           x3003: x126D  ADD R1, R1, #13                           [line: 5]
           x3004: x4845  JSR GETCHOICE                             [line: 6]
           x3005: xF022  PUTS                                      [line: 7]
           x3006: x5020  AND R0, R0, #0                            [line: 8]
           x3007: xF023  IN                                        [line: 9]
           x3008: x4844  JSR GETSTEP                               [line: 10]
           x3009: x1B60  ADD R5, R5, #0                            [line: 11]
           x300A: x03F9  BRp x3004                                 [line: 12]
           x300B: x481B  JSR POP                                   [line: 13]
           x300C: x2E3B  LD R7, REG7               